In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import torch
import torch.nn as nn
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

c:\Users\zhang\anaconda3\envs\tensorflow\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Data Generation

In [2]:
NUM_USER = 10000

In [3]:
NUM_Product = 4

In [4]:
treatment_percentage = 0.5

In [5]:
discount = 0.2

In [6]:
user_continuous_feature_multiplier = 1

In [7]:
prod_continuous_feature_multiplier = 1

In [8]:
# Set constants
USER_Cont_FEATURES = 2*user_continuous_feature_multiplier
USER_Dicr_FEATURES = 3

Product_Cont_FEATURES = 3*prod_continuous_feature_multiplier
Product_Dicr_FEATURES = 2
OUTSIDE_OPTION_UTILITY = 0
utilities = torch.zeros(NUM_USER, NUM_Product)

In [9]:
def generate_features(N, C, D):
    continuous_features = np.zeros((N, C))
    for i in range(C):
        continuous_features[:, i] = np.random.uniform(0,1,size=N)
    binary_features = np.random.randint(0, 2, (N, D))
    return np.hstack((continuous_features, binary_features))

In [10]:
import torch.nn as nn
import torch.nn.init as init

class UtilityDNN(nn.Module):
    def __init__(self, user_features, product_features):
        super(UtilityDNN, self).__init__()
        self.fc1 = nn.Linear(user_features + product_features, 1)
        nn.init.constant_(self.fc1.bias, 0)
        nn.init.uniform_(self.fc1.weight, a=-0.0, b=0.5)
    def forward(self, x):
        x = self.fc1(x)
        return x

class PriceSensitivityDNN(nn.Module):
    def __init__(self, user_features):
        super(PriceSensitivityDNN, self).__init__()
        self.fc1 = nn.Linear(user_features,1)
        self.fc2 = nn.Linear(1, 1)

        nn.init.constant_(self.fc1.weight, 0)
        nn.init.constant_(self.fc1.bias, 0)
        nn.init.constant_(self.fc2.weight, 0)
    def forward(self, x):

        x = self.fc1(x)
        x = self.fc2(x)
        return torch.abs(x)


In [11]:
X_user = generate_features(NUM_USER,USER_Cont_FEATURES, USER_Dicr_FEATURES)
X_product = generate_features(NUM_Product, Product_Cont_FEATURES, Product_Dicr_FEATURES)
price = np.random.uniform(0.5 ,1, NUM_Product)

X_user = torch.from_numpy(X_user).float()
X_product = torch.from_numpy(X_product).float()
price = torch.from_numpy(price).float()
gumbel_dist = torch.distributions.Gumbel(0, 1)
gumbel_noise = gumbel_dist.sample((NUM_USER, NUM_Product))

In [12]:
price

tensor([0.7871, 0.7414, 0.5153, 0.5254])

In [13]:
pair_utility_model = UtilityDNN(X_user.shape[1], X_product.shape[1])
price_sensitivity_model = PriceSensitivityDNN(X_user.shape[1])

In [14]:
layer_weights = pair_utility_model.fc1.weight.data
layer_biases = pair_utility_model.fc1.bias.data

print("Weights:", layer_weights)
print("Biases:", layer_biases)

Weights: tensor([[0.1368, 0.4839, 0.3859, 0.4845, 0.2574, 0.4539, 0.2923, 0.2170, 0.3591,
         0.0578]])
Biases: tensor([0.])


In [15]:

new_biases = torch.from_numpy(np.array(1.0))

In [16]:
# Assign new weights and biases
with torch.no_grad():  # Avoid tracking this operation in the computation graph
    price_sensitivity_model.fc2.bias.copy_(new_biases)
    

In [17]:
layer_weights = price_sensitivity_model.fc2.weight.data
layer_biases = price_sensitivity_model.fc2.bias.data

print("Weights:", layer_weights)
print("Biases:", layer_biases)

Weights: tensor([[0.]])
Biases: tensor([1.])


In [18]:
num_bundles = 2 ** X_product.shape[0]
bundle_utilities = torch.randn(num_bundles)
def utility_model_batched_with_bundles(x_user, X_product, price, user_randomization, prod_randomization, pair_utility_model, price_sensitivity_model, gumbel_noise, bundle_utilities, batch_size=10):
    num_users = x_user.shape[0]
    num_products = X_product.shape[0]
    num_bundles = 2 ** num_products  # Total number of possible bundles
    decisions = torch.zeros(num_users, dtype=torch.long)  # Initialize decision array

    # Generate all possible bundles (binary representations)
    bundle_choices = torch.tensor(
    [[int(bit) for bit in np.binary_repr(i, width=num_products)] for i in range(num_bundles)],
    dtype=torch.float32)

    # Convert numpy arrays to tensors if necessary
    if isinstance(user_randomization, np.ndarray):
        user_randomization = torch.from_numpy(user_randomization).to(torch.bool)
    if isinstance(prod_randomization, np.ndarray):
        prod_randomization = torch.from_numpy(prod_randomization).to(torch.bool)
    if isinstance(price, np.ndarray):
        price = torch.from_numpy(price)

    # Compute price sensitivities outside the batch loop
    price_sensitivities = price_sensitivity_model(x_user)

    # Iterate over users in batches
    for i in range(0, num_users, batch_size):
        batch_end = min(i + batch_size, num_users)  # Define the end of the batch
        batch_indices = slice(i, batch_end)  # Slice for batch indexing

        # Repeat the product features and price for each user in the batch
        batch_user_features = x_user[batch_indices].unsqueeze(1).expand(-1, num_products, -1)
        batch_prod_features = X_product.unsqueeze(0).expand(batch_end - i, -1, -1)
        batch_price = price.unsqueeze(0).expand(batch_end - i, -1)

        # Handle treatment adjustments in batch
        batch_user_treatment = user_randomization[batch_indices].unsqueeze(1).expand(-1, num_products) == 1
        batch_prod_treatment = prod_randomization.unsqueeze(0).expand(batch_end - i, -1) == 1
        batch_adjusted_price = torch.where(batch_user_treatment | batch_prod_treatment, batch_price * discount, batch_price)

        # Combine user and product features
        combined_features = torch.cat((batch_user_features, batch_prod_features), dim=2)

        # Compute utilities for each user-product pair using the neural network in a batch
        utility_from_dnn = pair_utility_model(combined_features.view(-1, combined_features.shape[-1])).view(batch_end - i, num_products)

        # Compute price effect
        price_effect = price_sensitivities[batch_indices] * batch_adjusted_price
        product_utilities = utility_from_dnn - price_effect + gumbel_noise[batch_indices]

        # Calculate total bundle utilities by summing product utilities for each bundle and adding bundle-specific utilities
        total_bundle_utilities = torch.zeros(batch_end - i, num_bundles)
        for b in range(num_bundles):
            bundle_mask = bundle_choices[b]  # Binary mask for the current bundle
            bundle_utilities_sum = (product_utilities * bundle_mask).sum(dim=1)  # Sum of product utilities in the bundle
            total_bundle_utilities[:, b] = bundle_utilities_sum + bundle_utilities[b]  # Add bundle-specific utility

        # Find the bundle with the highest utility for each user
        max_utilities, chosen_bundles = torch.max(total_bundle_utilities, dim=1)

        # The empty bundle (index 0) represents the outside option (no products chosen)
        decisions[batch_indices] = torch.where(max_utilities > 0, chosen_bundles, torch.zeros_like(chosen_bundles))

    return decisions


In [19]:
def calculate_revenue_bundle(decisions, prices):
    total_revenue = 0.0
    num_products = prices.shape[0]

    # Iterate over each decision (bundle index) and calculate the total price of the chosen products
    for decision in decisions:
        if decision != 0:  # Check if the decision is not the outside option (empty bundle)
            # Convert the bundle index to a binary mask representing the products in the bundle
            bundle_mask = torch.tensor([int(x) for x in np.binary_repr(decision.item(), width=num_products)], dtype=torch.bool)
            # Sum the prices of the products included in the bundle
            total_revenue += prices[bundle_mask].sum().item()

    return total_revenue

# GTE

## All treated scenario: all products are discounted

In [20]:
user_randomization = np.random.choice([0,1], NUM_USER, p=[1, 0])
prod_randomization = np.random.choice([0,1], NUM_Product, p=[0, 1])

In [21]:
decisions_all_treat=utility_model_batched_with_bundles(X_user, X_product, price, user_randomization, prod_randomization,
                                                         pair_utility_model, price_sensitivity_model, gumbel_noise, bundle_utilities, batch_size=10)
print("Decisions per user (product index or -1 for outside option):\n", decisions_all_treat)

Decisions per user (product index or -1 for outside option):
 tensor([15, 14,  7,  ..., 15, 15, 15])


In [22]:
all_num_unique = torch.unique(decisions_all_treat).numel()
print(all_num_unique)

13


In [23]:
for i in range(-1,16):
    print(torch.sum(decisions_all_treat==i))

tensor(0)
tensor(0)
tensor(1)
tensor(7)
tensor(207)
tensor(11)
tensor(13)
tensor(0)
tensor(744)
tensor(188)
tensor(66)
tensor(1)
tensor(0)
tensor(455)
tensor(2)
tensor(989)
tensor(7316)


In [24]:
total_revenue_all_treated = calculate_revenue_bundle(decisions_all_treat, price*discount)
print(f"Total revenue from sales when all products are discounted: ${total_revenue_all_treated:.2f}")

Total revenue from sales when all products are discounted: $4664.38


## All control scenario: all products remain the original price

In [25]:
user_randomization = np.random.choice([0,1], NUM_USER, p=[1, 0])
prod_randomization = np.random.choice([0,1], NUM_Product, p=[1, 0])

decisions_all_control =utility_model_batched_with_bundles(X_user, X_product, price, user_randomization, prod_randomization,
                                                         pair_utility_model, price_sensitivity_model, gumbel_noise, bundle_utilities, batch_size=10)

print("Decisions per user (product index or -1 for outside option):\n", decisions_all_control)
total_revenue_all_control = calculate_revenue_bundle(decisions_all_control, price)
print(f"Total Revenue from Sales: ${total_revenue_all_control:.2f}")

Decisions per user (product index or -1 for outside option):
 tensor([15,  2,  4,  ..., 15, 15, 15])
Total Revenue from Sales: $19928.09


In [26]:
revenue_difference = total_revenue_all_treated - total_revenue_all_control
print(f"Revenue Difference (ALLTreated - ALLControl): ${revenue_difference:.2f}")
# print(f"Revenue Relative Difference (ALLTreated - ALLControl)/AllControl: {100*revenue_difference/total_revenue_all_control:.2f}%")

Revenue Difference (ALLTreated - ALLControl): $-15263.71


In [27]:
true = revenue_difference

## product randomization

In [28]:
def calculate_bundle_revenue(decisions, prices, prod_randomization):
    revenue_treated = 0.0
    revenue_control = 0.0
    num_products = prices.shape[0]

    # Iterate over each user's decision (bundle index)
    for decision in decisions:
        if decision != 0:  # If the user did not choose the outside option (empty bundle)
            # Convert the bundle index to a binary mask representing the products in the bundle
            bundle_mask = torch.tensor([int(x) for x in np.binary_repr(decision.item(), width=num_products)], dtype=torch.bool)

            # Iterate over products in the bundle
            for product_index in range(num_products):
                if bundle_mask[product_index]:  # If the product is included in the bundle
                    product_price = prices[product_index].item()
                    if prod_randomization[product_index]:  # Check if the product is in the treatment group
                        revenue_treated += product_price
                    else:  # The product is in the control group
                        revenue_control += product_price

    return revenue_treated, revenue_control


In [29]:
utilities = torch.zeros(NUM_USER, NUM_Product)
user_randomization = np.random.choice([0,1], NUM_USER, p=[1, 0])
prod_randomization = np.random.choice([0,1], NUM_Product, p=[1-treatment_percentage, treatment_percentage])
# prod_randomization = np.random.choice([0,1], NUM_Product, p=[1, ])
decisions_product_randomization =utility_model_batched_with_bundles(X_user, X_product, price, user_randomization, prod_randomization,
                                                         pair_utility_model, price_sensitivity_model, gumbel_noise, bundle_utilities, batch_size=10)

In [30]:
revenue_treated, revenue_control = calculate_bundle_revenue(decisions_product_randomization, price-price*(1-discount)*prod_randomization, prod_randomization)
naive = revenue_treated/treatment_percentage - revenue_control/(1-treatment_percentage)
print(f"Revenue from Treated Products: ${revenue_treated:.2f}")
print(f"Revenue from Control Products: ${revenue_control:.2f}")
print(f"Revenue Difference (Treated - Control) by naive DIM: ${naive:.2f}")
# print(f"Revenue Relative Difference (ALLTreated - ALLControl)/AllControl: {100*revenue_difference/revenue_control:.2f}%")

Revenue from Treated Products: $865.61
Revenue from Control Products: $16231.20
Revenue Difference (Treated - Control) by naive DIM: $-30731.19


## Prepare training and testing data given experiment data

In [31]:
X_user_1, X_user_2, decision_1, decision_2 = train_test_split(
X_user, decisions_product_randomization, test_size=1/2, random_state=3407)


In [32]:
train_set = {
    'features': X_user_1,
    'labels': decision_1
}

test_set = {
    'features': X_user_2,
    'labels': decision_2
}

# Flag to switch between training and test set
use_train_set = False  # Set to False for the test set

# Function to get the current active dataset
def get_active_dataset(use_train):
    return train_set if use_train else test_set
def get_test_dataset(use_train):
    return test_set if use_train else train_set
# Retrieve the current dataset based on the flag
current_dataset = get_active_dataset(use_train_set)
X_user_train = current_dataset['features']
decision_train = current_dataset['labels']
X_user_test = get_test_dataset(use_train_set)['features']
decision_test =  get_test_dataset(use_train_set)['labels']

In [33]:
bundle_choices = torch.tensor(
    [[int(bit) for bit in np.binary_repr(i, width=X_product.shape[0])] for i in range(2**X_product.shape[0])],
    dtype=torch.float32)

# use simple MNL structural model

In [34]:
class BundleMNLModel(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim, num_bundles):
        super(BundleMNLModel, self).__init__()
        self.beta_user = nn.Parameter(torch.randn(user_feature_dim))
        self.beta_product = nn.Parameter(torch.randn(product_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0))
        self.bundle_utilities = nn.Parameter(torch.randn(num_bundles))  # Bundle-specific utilities

    def forward(self, x_user, X_product, price, prod_randomization, bundle_choices):
        N, M = x_user.shape[0], X_product.shape[0]
        num_bundles = bundle_choices.shape[0]

        # Expand user and product features to create a [N, M, F] shaped tensor
        x_user_expanded = x_user.unsqueeze(1).expand(-1, M, -1)
        X_product_expanded = X_product.unsqueeze(0).expand(N, -1, -1)

        # Calculate linear utility from features
        utility_user = torch.sum(x_user_expanded * self.beta_user, dim=2)
        utility_product = torch.sum(X_product_expanded * self.beta_product, dim=2)

        # Adjust prices based on product randomization (apply discount if treated)
        adjusted_price = torch.where(prod_randomization.unsqueeze(0), price * discount, price)
        utility_price = adjusted_price * self.beta_price

        # Total product utilities (user + product + price)
        product_utilities = utility_user + utility_product + utility_price

        # Calculate total bundle utilities for each user and bundle
        total_bundle_utilities = torch.zeros(N, num_bundles, device=x_user.device)
        for b in range(num_bundles):
            bundle_mask = bundle_choices[b]
            bundle_mask = bundle_mask.to(device)
            bundle_product_utilities = (product_utilities * bundle_mask).sum(dim=1)
            total_bundle_utilities[:, b] = bundle_product_utilities + self.bundle_utilities[b]


        return total_bundle_utilities

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_user_train, X_product, price = X_user_train.to(device), X_product.to(device), price.to(device)
if isinstance(user_randomization, np.ndarray):
    user_randomization = torch.from_numpy(user_randomization).to(X_user_train.device).bool()
if isinstance(prod_randomization, np.ndarray):
    prod_randomization = torch.from_numpy(prod_randomization).to(X_user_train.device).bool()

decision_train = decision_train.long().to(device)

In [36]:
import torch.optim as optim
model = BundleMNLModel(user_feature_dim=USER_Cont_FEATURES+USER_Dicr_FEATURES,
                       product_feature_dim=Product_Cont_FEATURES+Product_Dicr_FEATURES,
                       num_bundles = 2 ** NUM_Product).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [37]:
# Training loop
import torch.nn.functional as F
num_epochs = 2000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    utilities = model(X_user_train, X_product, price, prod_randomization, bundle_choices)
    choice_probabilities = F.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), decision_train])
    loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')




Epoch 0, Loss: 4.7495
Epoch 500, Loss: 1.4726
Epoch 1000, Loss: 1.4338
Epoch 1500, Loss: 1.4303


In [38]:
choice_probabilities

tensor([[-12.2344,  -7.1493,  -7.1129,  ...,  -7.0896,  -2.7675,  -0.4553],
        [-12.5081,  -7.3446,  -7.3082,  ...,  -7.1281,  -2.8061,  -0.4155],
        [-10.7963,  -6.1452,  -6.1088,  ...,  -6.9536,  -2.6316,  -0.7534],
        ...,
        [-10.3872,  -5.8690,  -5.8326,  ...,  -6.9434,  -2.6213,  -0.8761],
        [-12.3980,  -7.2659,  -7.2295,  ...,  -7.1123,  -2.7902,  -0.4310],
        [ -8.3438,  -4.6073,  -4.5709,  ...,  -7.2449,  -2.9228,  -1.9592]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward0>)

In [39]:
beta_price_est = model.beta_price.cpu().detach().numpy()

In [40]:
print(beta_price_est)

0.045324706


In [41]:
bundle_prices = torch.zeros(2**NUM_Product, device=device)
for b in range(2**NUM_Product):
    bundle_mask = bundle_choices[b]
    bundle_prices[b] = torch.sum(price[bundle_mask.bool()])

In [42]:

# Control group (all products are control)
all_product_control = torch.zeros(NUM_Product, dtype=torch.bool).to(device)
# Treated group (all products are treated)
all_product_treated = torch.ones(NUM_Product, dtype=torch.bool).to(device)

X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

# Calculate expected revenue for control group
utilities_control = model(X_user_test, X_product, price, all_product_control, bundle_choices)
probabilities_control = F.softmax(utilities_control, dim=1)
expected_revenue_control = torch.sum(probabilities_control * bundle_prices.expand_as(probabilities_control), dim=0).sum()

print(f"Expected Revenue (Control Group): ${expected_revenue_control.item():.2f}")

# Calculate expected revenue for treated group
utilities_treated = model(X_user_test, X_product, price, all_product_treated, bundle_choices)
probabilities_treated = F.softmax(utilities_treated, dim=1)
expected_revenue_treated = torch.sum(probabilities_treated * bundle_prices.expand_as(probabilities_treated), dim=0).sum()

print(f"Expected Revenue (Treated Group): ${expected_revenue_treated.item():.2f}")

linear = (expected_revenue_treated-expected_revenue_control).cpu().detach().numpy()
linear = linear*2


Expected Revenue (Control Group): $10270.00
Expected Revenue (Treated Group): $10213.36


In [43]:

print(f"Revenue Difference (Treated - Control) by Linear MNL: ${linear:.2f}")
print(f"Absolute Percentage Estimation Error of Linear MNL:  {100*np.abs(linear-revenue_difference)/revenue_difference:.2f}%")


Revenue Difference (Treated - Control) by Linear MNL: $-113.28
Absolute Percentage Estimation Error of Linear MNL:  -99.26%


# use PDL

In [44]:
X_user_train1, X_user_val, decision_train1,decision_val = train_test_split(X_user_train,decision_train,test_size=0.1,random_state=34)

In [45]:
import torch
import numpy as np

def prepare_data_bundle(user_features, product_features, prices):
    # distinct device reference
    device = product_features.device
    
    num_products = product_features.shape[0]
    num_bundles = 2 ** num_products
    bundle_choices = torch.tensor(
        [[int(bit) for bit in np.binary_repr(i, width=num_products)] for i in range(num_bundles)],
        dtype=torch.bool,
        device=device 
    )
    
    # Calculate bundle prices
    bundle_prices = torch.tensor([prices[bundle_mask].sum() for bundle_mask in bundle_choices], device=device)

    # Initialize lists
    all_x_included_products = []
    all_x_other_products = []
    all_bundle_prices = []
    
    # Iterate through each bundle
    for i, bundle_mask in enumerate(bundle_choices):
        # Get included product indices
        included_indices = torch.where(bundle_mask)[0]
        excluded_indices = torch.where(~bundle_mask)[0]

        if included_indices.nelement() > 0:
            included_products = product_features[included_indices].reshape(-1)
        else:
            included_products = torch.tensor([], dtype=product_features.dtype, device=device)

        # Features of excluded products
        if excluded_indices.nelement() > 0:
            other_products = product_features[excluded_indices].reshape(-1)
        else:
            other_products = torch.tensor([], dtype=product_features.dtype, device=device)

        # Price of the current bundle
        current_bundle_price = bundle_prices[i]

        # Append to lists
        all_x_included_products.append(included_products)
        all_x_other_products.append(other_products)
        all_bundle_prices.append(current_bundle_price)
        

    max_included_len = max([x.numel() for x in all_x_included_products])
    max_other_len = max([x.numel() for x in all_x_other_products])

  
    all_x_included_products = torch.stack([
        torch.cat([
            x, 
            torch.zeros(max_included_len - x.numel(), device=device, dtype=x.dtype)
        ]) for x in all_x_included_products
    ])

    all_x_other_products = torch.stack([
        torch.cat([
            x, 
            torch.zeros(max_other_len - x.numel(), device=device, dtype=x.dtype)
        ]) for x in all_x_other_products
    ])
    
    all_bundle_prices = torch.stack(all_bundle_prices)

    return user_features, product_features, prices, bundle_choices, all_x_included_products, all_x_other_products, all_bundle_prices

In [46]:
price = price.to(device)
X_user_train1 = X_user_train1.to(device)
#for complementarity model
prepared_data = prepare_data_bundle(X_user_train1, X_product,  price * (1 - (1-discount) * prod_randomization))
user_features, product_features, prices, bundle_choices, all_x_included_products, all_x_other_products, all_bundle_prices= prepared_data

In [47]:
class PDLModel(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim):
        super(PDLModel, self).__init__()
        # Combined feature dimension includes product features, price, and user features, as well as other products' features and prices
        total_feature_dim = user_feature_dim +product_feature_dim*(NUM_Product)+2  # +1 for price

        # Single neural network to process the combined features
        self.network = nn.Sequential(
            nn.Linear(total_feature_dim, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5,5),
            nn.ReLU(),
            nn.Linear(5, 1) 
        )
            # Layers to process other products' features (z-j)
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_feature_dim*(NUM_Product), NUM_Product),
            nn.ReLU(),
            nn.Linear(NUM_Product, 1)
        )

    def forward(self, x_user, x_product, x_other_products,prices):
        N = x_user.shape[0]
        M = x_product.shape[0]
        # Process other products' features
        aggregated_other_features = self.other_product_features_layers(x_other_products)

        
        combined_features =  torch.cat((x_user.unsqueeze(1).expand(-1, M, -1),
                                        x_product.unsqueeze(0).expand(N, -1, -1),
                                        aggregated_other_features.unsqueeze(0).expand(N, -1, -1),
                                        prices.view(1, -1, 1).expand(N, -1, -1)),
                                        dim=2)
   

        # Compute utility for each combined feature set
        utilities = self.network(combined_features).squeeze(-1)


        return utilities
        
   

In [48]:
pdlmodel = PDLModel(user_feature_dim=USER_Cont_FEATURES+USER_Dicr_FEATURES,
                       product_feature_dim=Product_Cont_FEATURES+Product_Dicr_FEATURES).to(device)

In [49]:

optimizer = torch.optim.Adam(pdlmodel.parameters(), lr=0.01)

best_val_loss = float('inf')
patience = 15
patience_counter = 0

for epoch in range(1000):
    pdlmodel.train()  # Set model to training mode
    optimizer.zero_grad()


    outputs = pdlmodel(user_features, all_x_included_products, all_x_other_products,all_bundle_prices)
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), decision_train1])

    loss.backward()
    optimizer.step()

    # Validation phase
    pdlmodel.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = pdlmodel(X_user_val,  all_x_included_products, all_x_other_products ,all_bundle_prices)
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]),decision_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break


Epoch 1, Training Loss: 2.7710773944854736, Validation Loss: 2.762986421585083
Epoch 2, Training Loss: 2.763418436050415, Validation Loss: 2.7556018829345703
Epoch 3, Training Loss: 2.7562291622161865, Validation Loss: 2.747253656387329
Epoch 4, Training Loss: 2.7483325004577637, Validation Loss: 2.7366998195648193
Epoch 5, Training Loss: 2.7381398677825928, Validation Loss: 2.7237296104431152
Epoch 6, Training Loss: 2.7255008220672607, Validation Loss: 2.709301471710205
Epoch 7, Training Loss: 2.711395502090454, Validation Loss: 2.6926827430725098
Epoch 8, Training Loss: 2.695099353790283, Validation Loss: 2.6736888885498047
Epoch 9, Training Loss: 2.6766371726989746, Validation Loss: 2.6526615619659424
Epoch 10, Training Loss: 2.656370162963867, Validation Loss: 2.629607915878296
Epoch 11, Training Loss: 2.634136915206909, Validation Loss: 2.6068639755249023
Epoch 12, Training Loss: 2.6125316619873047, Validation Loss: 2.586130142211914
Epoch 13, Training Loss: 2.5927205085754395, Va

In [50]:
#for complementarity model
def calculate_expected_revenue(model, user_features, all_x_included_products, bundle_prices):
    # Ensure model is in evaluation mode
    model.eval()

    with torch.no_grad():  # Disable gradient calculation
        # Calculate utilities for all bundles
        utilities = model(user_features, all_x_included_products, all_x_other_products,bundle_prices)
        probabilities = F.softmax(utilities, dim=1)

        # Calculate total expected revenue
        total_expected_revenue = (probabilities * bundle_prices.unsqueeze(0)).sum()

    return total_expected_revenue.item()  # Convert to Python float


In [51]:
import torch.nn.functional as F
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)
control_prepared_data = prepare_data_bundle(X_user_test, X_product,  price)
user_features, product_features, prices, bundle_choices, all_x_included_products, all_x_other_products,all_bundle_prices = control_prepared_data
expected_revenue_all_control = calculate_expected_revenue(pdlmodel, user_features, all_x_included_products, all_bundle_prices, )
print(f"Expected Revenue all Control: ${expected_revenue_all_control:.2f}")

all_treated_price = price*discount
treated_prepared_data = prepare_data_bundle(X_user_test, X_product,  all_treated_price)
user_features, product_features, prices, bundle_choices, all_x_included_products, all_x_other_products, all_bundle_prices = treated_prepared_data
expected_revenue_all_treated = calculate_expected_revenue(pdlmodel, user_features, all_x_included_products, all_bundle_prices)
print(f"Expected Revenue all treated: ${expected_revenue_all_treated:.2f}")
expected_revenue_all_treated-expected_revenue_all_control

Expected Revenue all Control: $10209.66
Expected Revenue all treated: $2104.48


-8105.18017578125

In [52]:
pdl = (expected_revenue_all_treated-expected_revenue_all_control)*2

In [53]:
print(f"Absolute Percentage Estimation Error of PDL:  {100*np.abs(pdl-revenue_difference)/revenue_difference:.2f}%")

Absolute Percentage Estimation Error of PDL:  -6.20%


# use dml

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_user_train, X_product, price = X_user_train.to(device), X_product.to(device), price.to(device)
if isinstance(user_randomization, np.ndarray):
    user_randomization = torch.from_numpy(user_randomization).to(X_user_train.device).bool()
if isinstance(prod_randomization, np.ndarray):
    prod_randomization = torch.from_numpy(prod_randomization).to(X_user_train.device).bool()

decision_train = decision_train.long().to(device)

In [55]:
class UtilityEstimator(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim):
        super(UtilityEstimator, self).__init__()
        
        # Layers to process other products' features (z-j)
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_feature_dim*(NUM_Product), NUM_Product),
            nn.ReLU(),
            nn.Linear(NUM_Product, 1)
        )

        self.theta0 = nn.Sequential(
            nn.Linear(user_feature_dim +product_feature_dim*(NUM_Product)+1, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
        )
       
        self.theta1 = nn.Sequential(
            nn.Linear(user_feature_dim + product_feature_dim*(NUM_Product)+1, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
        )
        
    def forward(self, x_user, x_product, x_other_products,price):
        N = x_user.shape[0]
        M = x_product.shape[0]
        aggregated_other_features = self.other_product_features_layers(x_other_products)

        
        combined_features_theta =  torch.cat((x_user.unsqueeze(1).expand(-1, M, -1),
                                               x_product.unsqueeze(0).expand(N, -1, -1),
                                               aggregated_other_features.unsqueeze(0).expand(N, -1, -1)),
                                                 dim=2)
        theta0_output = self.theta0(combined_features_theta).squeeze(-1)
        theta1_output = self.theta1(combined_features_theta).squeeze(-1)
        
        price = price.unsqueeze(-1)  
        utility = theta0_output + theta1_output * price.squeeze(-1)


        
        return utility,theta0_output,theta1_output


In [56]:
dml_model = UtilityEstimator(user_feature_dim=USER_Cont_FEATURES+USER_Dicr_FEATURES,
                       product_feature_dim=Product_Cont_FEATURES+Product_Dicr_FEATURES).to(device)

In [57]:
X_user_train1, X_user_val, decision_train1,decision_val = train_test_split(X_user_train,decision_train,test_size=0.1,random_state=34)

In [58]:
import torch
import numpy as np

def prepare_data_bundle(user_features, product_features, prices):
    # distinct device reference
    device = product_features.device
    
    num_products = product_features.shape[0]
    num_bundles = 2 ** num_products
    bundle_choices = torch.tensor(
        [[int(bit) for bit in np.binary_repr(i, width=num_products)] for i in range(num_bundles)],
        dtype=torch.bool,
        device=device 
    )
    
    # Calculate bundle prices
    bundle_prices = torch.tensor([prices[bundle_mask].sum() for bundle_mask in bundle_choices], device=device)

    # Initialize lists
    all_x_included_products = []
    all_x_other_products = []
    all_bundle_prices = []
    
    # Iterate through each bundle
    for i, bundle_mask in enumerate(bundle_choices):
        # Get included product indices
        included_indices = torch.where(bundle_mask)[0]
        excluded_indices = torch.where(~bundle_mask)[0]

        if included_indices.nelement() > 0:
            included_products = product_features[included_indices].reshape(-1)
        else:
            included_products = torch.tensor([], dtype=product_features.dtype, device=device)

        # Features of excluded products
        if excluded_indices.nelement() > 0:
            other_products = product_features[excluded_indices].reshape(-1)
        else:
            other_products = torch.tensor([], dtype=product_features.dtype, device=device)

        # Price of the current bundle
        current_bundle_price = bundle_prices[i]

        # Append to lists
        all_x_included_products.append(included_products)
        all_x_other_products.append(other_products)
        all_bundle_prices.append(current_bundle_price)
        

    max_included_len = max([x.numel() for x in all_x_included_products])
    max_other_len = max([x.numel() for x in all_x_other_products])

  
    all_x_included_products = torch.stack([
        torch.cat([
            x, 
            torch.zeros(max_included_len - x.numel(), device=device, dtype=x.dtype)
        ]) for x in all_x_included_products
    ])

    all_x_other_products = torch.stack([
        torch.cat([
            x, 
            torch.zeros(max_other_len - x.numel(), device=device, dtype=x.dtype)
        ]) for x in all_x_other_products
    ])
    
    all_bundle_prices = torch.stack(all_bundle_prices)

    return user_features, product_features, prices, bundle_choices, all_x_included_products, all_x_other_products, all_bundle_prices

In [59]:
price = price.to(device)
X_user_train1 = X_user_train1.to(device)
#for complementarity model
prepared_data = prepare_data_bundle(X_user_train1, X_product,  price * (1 - (1-discount) * prod_randomization))
user_features, product_features, prices, bundle_choices, all_x_included_products, all_x_other_products, all_bundle_prices= prepared_data

In [60]:
import torch.nn.functional as F
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.01)

best_val_loss = float('inf')
patience = 10
patience_counter = 0


for epoch in range(1000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()


    outputs = dml_model(user_features, all_x_included_products, all_x_other_products,all_bundle_prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), decision_train1])

    loss.backward()
    optimizer.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val,  all_x_included_products, all_x_other_products ,all_bundle_prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]),decision_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break


Epoch 1, Training Loss: 2.9187653064727783, Validation Loss: 2.8863701820373535
Epoch 2, Training Loss: 2.8834474086761475, Validation Loss: 2.852341651916504
Epoch 3, Training Loss: 2.8501596450805664, Validation Loss: 2.8190836906433105
Epoch 4, Training Loss: 2.817775011062622, Validation Loss: 2.783125638961792
Epoch 5, Training Loss: 2.782428026199341, Validation Loss: 2.743191957473755
Epoch 6, Training Loss: 2.7428579330444336, Validation Loss: 2.698638439178467
Epoch 7, Training Loss: 2.698559522628784, Validation Loss: 2.649301767349243
Epoch 8, Training Loss: 2.65019154548645, Validation Loss: 2.595186233520508
Epoch 9, Training Loss: 2.597923755645752, Validation Loss: 2.5369913578033447
Epoch 10, Training Loss: 2.54122257232666, Validation Loss: 2.4749250411987305
Epoch 11, Training Loss: 2.481414794921875, Validation Loss: 2.412259340286255
Epoch 12, Training Loss: 2.420891046524048, Validation Loss: 2.351177453994751
Epoch 13, Training Loss: 2.3618080615997314, Validation

In [61]:
#for complementarity model
def calculate_expected_revenue(model, user_features, all_x_included_products, bundle_prices):
    # Ensure model is in evaluation mode
    model.eval()

    with torch.no_grad():  # Disable gradient calculation
        # Calculate utilities for all bundles
        utilities = model(user_features, all_x_included_products, all_x_other_products,bundle_prices)[0]
        probabilities = F.softmax(utilities, dim=1)

        # Calculate total expected revenue
        total_expected_revenue = (probabilities * bundle_prices.unsqueeze(0)).sum()

    return total_expected_revenue.item()  # Convert to Python float


In [62]:
import torch.nn.functional as F
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)
control_prepared_data = prepare_data_bundle(X_user_test, X_product,  price)
user_features, product_features, prices, bundle_choices, all_x_included_products, all_x_other_products, all_bundle_prices = control_prepared_data
expected_revenue_all_control = calculate_expected_revenue(dml_model, user_features, all_x_included_products, all_bundle_prices)
print(f"Expected Revenue all Control: ${expected_revenue_all_control:.2f}")

all_treated_price = price*discount
treated_prepared_data = prepare_data_bundle(X_user_test, X_product,  all_treated_price)
user_features, product_features, prices, bundle_choices, all_x_included_products, all_x_other_products, all_bundle_prices = treated_prepared_data
expected_revenue_all_treated = calculate_expected_revenue(dml_model, user_features, all_x_included_products, all_bundle_prices)
print(f"Expected Revenue all treated: ${expected_revenue_all_treated:.2f}")
expected_revenue_all_treated-expected_revenue_all_control

Expected Revenue all Control: $10135.72
Expected Revenue all treated: $1638.34


-8497.37890625

In [63]:
expected_revenue_all_treated-expected_revenue_all_control

-8497.37890625

# debias the GTE estimator:

In [64]:
test_prepared_data = prepare_data_bundle(X_user_test, X_product,  price*(1-(1-discount)*prod_randomization))
user_features, product_features, prices, bundle_choices, all_x_included_products, all_x_other_products, all_bundle_prices = test_prepared_data

# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, all_x_included_products, all_x_other_products,all_bundle_prices)


# use formulation debias for H_i

In [65]:
def H_theta(theta0_output,theta1_output,all_treated_price,price):
    N = theta0_output.shape[0]
    M = 2**NUM_Product
    expand_price = price.unsqueeze(0).expand(N, M)
    expand_all_treated_price = all_treated_price.unsqueeze(0).expand(N, M)
    all_treated_uti = theta0_output + theta1_output * expand_all_treated_price
    all_control_uti =  theta0_output + theta1_output * expand_price


    # Include the outside option (utility = 0)
    zero_utilities = torch.zeros(N, 1, device=all_treated_uti.device)
    all_treated_uti = torch.cat((zero_utilities,all_treated_uti), dim=1)
    all_control_uti = torch.cat((zero_utilities,all_control_uti), dim=1)

    all_treated_probabilities = F.softmax(all_treated_uti, dim=1)
    all_control_probabilities = F.softmax(all_control_uti, dim=1)

    price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
    treated_price_with_outside =  torch.cat((torch.zeros(1, device=all_treated_price.device),all_treated_price), dim=0)

    H = torch.sum(all_treated_probabilities*treated_price_with_outside - all_control_probabilities*price_with_outside,dim=1)
    expsum_treated = torch.sum(torch.exp(all_treated_uti),dim=1)
    expsum_control = torch.sum(torch.exp(all_control_uti),dim=1)

    expsum_treated_expanded = expsum_treated.unsqueeze(1).expand(-1, all_treated_uti.shape[1])  # Shape [N, M+1]
    expsum_control_expanded = expsum_control.unsqueeze(1).expand(-1, all_control_uti.shape[1])  # Shape [N, M+1]

    H_theta0 = torch.sum((torch.exp(all_treated_uti)*(1-torch.exp(all_treated_uti))/expsum_treated_expanded/expsum_treated_expanded-\
                          torch.exp(all_control_uti)*(1-torch.exp(all_control_uti))/expsum_control_expanded/expsum_control_expanded)\
                         *price_with_outside,dim=1)
    H_theta1 = torch.sum(price_with_outside*(torch.exp(all_treated_uti)*(1-torch.exp(all_treated_uti))/expsum_treated_expanded/expsum_treated_expanded*treated_price_with_outside-\
                                             torch.exp(all_control_uti)*(1-torch.exp(all_control_uti))/expsum_control_expanded/expsum_control_expanded*price_with_outside),dim=1)


    return H,H_theta0,H_theta1


In [66]:
all_treated_price = price*discount
all_treated_bundle_price = torch.zeros(2**NUM_Product, device=device)
for b in range(2**NUM_Product):
    bundle_mask = bundle_choices[b]
    all_treated_bundle_price[b] = torch.sum(all_treated_price[bundle_mask.bool()])
all_bundle_prices= torch.zeros(2**NUM_Product, device=device)
for b in range(2**NUM_Product):
    bundle_mask = bundle_choices[b]
    all_bundle_prices[b] = torch.sum(price[bundle_mask.bool()])
H,H_theta0,H_theta1 = H_theta(theta0_output,theta1_output,all_treated_bundle_price,all_bundle_prices)

In [67]:
def l_theta(theta0_output,theta1_output,adjusted_price,decision_test):
    N = theta0_output.shape[0]
    M = 2**NUM_Product
    expand_adjusted_price = adjusted_price.unsqueeze(0).expand(N, M)
    uti = theta0_output + theta1_output * expand_adjusted_price
    adjusted_price_with_outside =  torch.cat([torch.zeros(1, device=adjusted_price.device),adjusted_price])

    probabilities = F.softmax(uti, dim=1)
    prod_indices = torch.ones(2**NUM_Product, device=device)
    ltheta0 = probabilities[torch.arange(decision_test.size(0)), decision_test] -prod_indices[decision_test]
    ltheta1 = (probabilities[torch.arange(decision_test.size(0)), decision_test] * adjusted_price_with_outside[decision_test]) - adjusted_price_with_outside[decision_test]


    return ltheta0,ltheta1

In [68]:
price = price.to(device)

In [69]:
adjusted_price = price*(1-(1-discount)*prod_randomization).to(device)
adjusted_bundle_price = torch.zeros(2**NUM_Product, device=device)
for b in range(2**NUM_Product):
    bundle_mask = bundle_choices[b]
    adjusted_bundle_price[b] = torch.sum(adjusted_price[bundle_mask.bool()])
decision_test = decision_test.to(device)
ltheta0,ltheta1= l_theta(theta0_output,theta1_output,adjusted_bundle_price,decision_test)

In [70]:
import torch
import torch.nn.functional as F

def lambdainv(theta0_output, theta1_output, price, decision_test,epsilon =10):
    N = theta0_output.shape[0]
    M = 2**NUM_Product
    expand_price = price.unsqueeze(0).expand(N, M)
    expand_all_treated_price = discount*price.unsqueeze(0).expand(N, M)

    all_treated_uti = theta0_output + theta1_output * expand_all_treated_price
    all_control_uti =  theta0_output + theta1_output * expand_price


    # Calculate probabilities using softmax
    probabilities_control = F.softmax(all_control_uti, dim=1)
    probabilities_treated = F.softmax(all_treated_uti, dim=1)

    # Extract probabilities of chosen products
    chosen_prob_control = probabilities_control[torch.arange(N), decision_test]
    chosen_prob_treated = probabilities_treated[torch.arange(N), decision_test]

    # Calculate second derivatives
    ltheta00 = -chosen_prob_control * (1 - chosen_prob_control) - chosen_prob_treated * (1 - chosen_prob_treated)
    ltheta01 = -chosen_prob_control * (1 - chosen_prob_control) * expand_price[torch.arange(N), decision_test] - \
            chosen_prob_treated * (1 - chosen_prob_treated) * (discount * expand_price[torch.arange(N), decision_test])
    ltheta11 = -chosen_prob_control * (1 - chosen_prob_control) * expand_price[torch.arange(N), decision_test]**2 - \
            chosen_prob_treated * (1 - chosen_prob_treated) * (discount * expand_price[torch.arange(N), decision_test])**2
    ltheta00=ltheta00/2
    ltheta01=ltheta01/2
    ltheta11=ltheta11/2

    # Form the 2x2 Hessian matrices for each instance
    ltheta00 = ltheta00.unsqueeze(1).unsqueeze(2)
    ltheta01 = ltheta01.unsqueeze(1).unsqueeze(2)
    ltheta11 = ltheta11.unsqueeze(1).unsqueeze(2)

    top_row = torch.cat((ltheta00, ltheta01), dim=2)
    bottom_row = torch.cat((ltheta01, ltheta11), dim=2)

    L_matrix = torch.cat((top_row, bottom_row), dim=1)

    # Regularization and inversion

    identity_matrix = torch.eye(2, dtype=L_matrix.dtype, device=L_matrix.device) * epsilon
    L_matrix_reg = L_matrix + identity_matrix.unsqueeze(0).unsqueeze(0)
    L_inv = torch.linalg.inv(L_matrix_reg)

    return L_inv


In [71]:
epsilon_list = [0.001,0.01,0.1,0.5,1,5,10]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, all_bundle_prices, decision_test, epsilon).float()

        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

        # Calculate sdl and dedl
        sdl = H.sum().cpu().detach().numpy() * 2
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 2

        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)

        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

In [72]:
sdl = H.sum().cpu().detach().numpy()*2

In [73]:
dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy())*2

In [74]:
sdl,dedl,best_epsilon

(-16983.65625, -14962.857421875, 10)

In [75]:
print(f"Absolute Percentage Estimation Error of SDL:  {100*np.abs(sdl-revenue_difference)/revenue_difference:.2f}%")
print(f"Absolute Percentage Estimation Error of SP MNL:  {100*np.abs(dedl-revenue_difference)/revenue_difference:.2f}%")

Absolute Percentage Estimation Error of SDL:  -11.27%
Absolute Percentage Estimation Error of SP MNL:  -1.97%


In [76]:
naive_pe = (naive - true) / true
linear_pe = (linear - true) / true
pdl_pe = (pdl - true) / true
sdl_pe = (sdl - true) / true
dedl_pe = (dedl - true) / true
naive_mse = (naive - true)**2
linear_mse =(linear - true)**2
pdl_mse = (pdl - true)**2
sdl_mse = (sdl - true)**2
dedl_mse = (dedl - true)**2
naive_e = (naive - true)
linear_e =(linear - true)
pdl_e = (pdl - true)
sdl_e = (sdl - true)
dedl_e = (dedl - true)

In [77]:
print(naive_pe,linear_pe,pdl_pe,sdl_pe,dedl_pe,naive_e,linear_e,pdl_e,sdl_e,dedl_e,naive_mse,linear_mse,pdl_mse,sdl_mse,dedl_mse)

1.0133498693682532 -0.9925782646659882 0.062019795339858574 0.1126821824789937 -0.019710208024618734 -15467.476799160242 15150.425084322691 -946.652064114809 -1719.947962552309 300.85086557269096 239242838.53256038 229535380.23567423 896150.1304928286 2958220.993887839 90511.24331583738
